[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Machine Learning Methods

## Exercise 002 - Classification

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 30/01/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/MachineLearningMethods/2023_01/Exercise0002ClassificationSolution.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, make_scorer, roc_auc_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

# Miscellaneous
import os
from platform import python_version
import random
import urllib.request

# Load MAT Files
from scipy.io import loadmat

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF = (8, 8)
ELM_SIZE_DEF = 50
CLASS_COLOR = ('b', 'r')
EDGE_COLOR  = 'k'

TEST_DATA_FILE_NAME  = 'TestData.mat'
TRAIN_DATA_FILE_NAME = 'TrainData.mat'

L_CLASSES   = ['Red', 'Green', 'Blue']
IMG_SIZE    = [100, 100]


In [ ]:
# Fixel Algorithms Packages


## Exercise

This exercise introduces:

 - Concept of _Features Transform_ to reduce the amount of data.
 - Optimizing a classifier by the accuracy score.


* <font color='brown'>(**#**)</font> While in this case the _dimensionality reduction_ of data is done manually by domain knowledge, later in the course we'll learn ML based methods.
* <font color='brown'>(**#**)</font> There is more than one way to implement the exercise. Feel free to wander.

In this exercise we'll work on images which are composed in a matrix.  
Each image is of size `100 x 100 x 3` yet it is spread in a _column stack_ fashion as a row in the data matrix.  
The train data has `2700` images in a matrix, so the matrix size is `2_700 x 30_000`.

The objective is being able to classify the color of the image: `Red: 0`, `Green: 1`, `Blue: 2`.  
The image colors isn't uniform but contains many colors, but the idea is to identify the dominant color.  
The concept of _red_ / _green_ / _blue_ is not a single color but the _family_ of colors.

1. Download the data `zip` file from https://drive.google.com/file/d/17-IWjWCPuXMSO0uUWKVDIO-NT38SoB8J.  
   Unzip it to the **same folder as this notebook**.
2. Extract features from the images for the training data.
3. Train a Kernel SVM (`rbf`) model on the data. Optimize the `C` and `gamma` parameters for accuracy using grid search.
4. Extract the same features from the test images into test data.
5. Plot the _confusion matrix_ of the best model on the test data.

Optimize features (repeat if needed) to get accuracy of at least `85%` per class.

In [ ]:
# Parameters

#===========================Fill This===========================#
# 1. Think of the parameters to optimize per model (See above).
# 2. Select the set to optimize over.
# 3. Set the number of folds in the cross validation.
lC = [0.1, 0.5, 1, 3]
lγ = ['scale', 'auto', 0.05, 0.5, 1.00, 3.00]
numFold = 5
#===============================================================#

In [ ]:
# Auxiliary Functions

def PlotImage(vX, imgClass = None, imgSize = IMG_SIZE, hA = None):

    mI = np.reshape(vX, (imgSize[0], imgSize[1], 3), order = 'F') #<! Data is coming from MATLAB

    if hA is None:
        hF, hA = plt.subplots(figsize = (4, 4))
    
    hA.imshow(mI)
    hA.tick_params(axis = 'both', left = False, top = False, right = False, bottom = False, labelleft = False, labeltop = False, labelright = False, labelbottom = False)

    if imgClass is not None:
        hA.set_title('Image Class: {imgClass}')

    return hA


def PlotImages(mX: np.ndarray, vY: np.ndarray, numRows: int, numCols: int, lClass = L_CLASSES, hF = None):

    numSamples  = mX.shape[0]
    numPx       = mX.shape[1]

    numImg = numRows * numCols

    tFigSize = (numRows * 3, numCols * 3)

    if hF is None:
        hF, hA = plt.subplots(nrows = numRows, ncols = numCols, figsize = tFigSize)
    else:
        hA = hF.axis
    
    hA = np.atleast_1d(hA) #<! To support numImg = 1
    hA = hA.flat

    vIdx = np.random.choice(numSamples, numImg, replace = False)
    
    for kk in range(numImg):
        imgIdx  = vIdx[kk]

        PlotImage(mX[imgIdx], hA = hA[kk])
        hA[kk].set_title(f'Index = {imgIdx}, Label = {lClass[vY[imgIdx]]}')
    
    plt.show()

def PlotLabelsHistogram(vY: np.ndarray, hA = None, lClass = None):

    if hA is None:
        hF, hA = plt.subplots(figsize = (8, 6))
    
    vLabels, vCounts = np.unique(vY, return_counts = True)

    hA.bar(vLabels, vCounts, width = 0.9, align = 'center')
    hA.set_title('Histogram of Classes / Labels')
    hA.set_xlabel('Class')
    hA.set_ylabel('Number of Samples')
    hA.set_xticks(vLabels)
    if lClass is not None:
        hA.set_xticklabels(lClass)

    return hA

def PlotConfusionMatrix(vY: np.ndarray, vYPred: np.ndarray, normMethod: str = None, hA: plt.Axes = None, lLabels: list = None, dScore: dict = None, titleStr: str = 'Confusion Matrix') -> plt.Axes:

    # Calculation of Confusion Matrix
    mConfMat = confusion_matrix(vY, vYPred, normalize = normMethod)
    oConfMat = ConfusionMatrixDisplay(mConfMat, display_labels = lLabels)
    oConfMat = oConfMat.plot(ax = hA)
    hA = oConfMat.ax_
    if dScore is not None:
        titleStr += ':'
        for scoreName, scoreVal in  dScore.items():
            titleStr += f' {scoreName} = {scoreVal:0.2},'
        titleStr = titleStr[:-1]
    hA.set_title(titleStr)
    hA.grid(False)

    return hA, mConfMat
    

## Generate / Load Data


In [ ]:
# Loading / Generating Data

dTestData  = loadmat(TEST_DATA_FILE_NAME)
dTrainData = loadmat(TRAIN_DATA_FILE_NAME)

mXTrain, vYTrain    = dTrainData['mX'], np.squeeze(dTrainData['vY'])
mXTest, vYTest      = dTestData['mX'], np.squeeze(dTestData['vY'])


In [ ]:
# Dimensions of the Data
print(f'The number of training data samples: {mXTrain.shape[0]}')
print(f'The number of training features per sample: {mXTrain.shape[1]}') 


print(f'The number of test data samples: {mXTest.shape[0]}')
print(f'The number of test features per sample: {mXTest.shape[1]}') 

### Plot Data

In [ ]:
# Display the Data (Some Images)

# Train Data
PlotImages(mXTrain, vYTrain, 3, 3)

In [ ]:
# Display the Data (Some Images)

# Test Data
PlotImages(mXTest, vYTest, 3, 3)

In [ ]:
# Histogram of Classes

# Train
hA = PlotLabelsHistogram(vYTrain, lClass = L_CLASSES)
hA.set_title(hA.get_title() + ' - Train Data')
plt.show()

In [ ]:
# Histogram of Classes

# Test
hA = PlotLabelsHistogram(vYTest, lClass = L_CLASSES)
hA.set_title(hA.get_title() + ' - Test Data')
plt.show()

* <font color='red'>(**?**)</font> Is the data balanced or imbalanced?

## Training Data and Feature Engineering / Extraction

The vector of values doesn't fit, as is, for classification with SVM.  
It misses a lot of the information given in the structure of the image or a color pixel.  
In our case, the important thing is to give the classifier information about the structure of color, a vector of 3 values: `[r, g, b]`.  
Yet, the classifier input is limited to a list of values. This is where the concept of metric comes into play.  

We need to create information about distance between colors.  
We also need to extract features to represent the colors in the image.

In this section the task are:

1. Implement functions to extract features from the data.
2. Arrange the features in a _matrix_ / _data frame_ for processing.
3. Explore the features using _SeaBorn_. Specifically if the features extracts meaningful information.

* <font color='brown'>(**#**)</font> Don't include _test data_ in the analysis for feature extraction. Other wise, a data leakage will happen.

### Ideas for Features

1. The distance between the the _mean_ / _median_ / _mode_ color of the image to the per _mean_ / _median_ / _mode_ color per class.
2. The distance between the quantized histogram of `R` / `G` / `B` color channels of the image to the class.
3. The distance of the mean color at the center of the image to the mean color of the class.
4. The channel with the maximum value (Is this a continuous value? Does it fit the SVM model?).
5. Use of the _HSL_ color space.


* <font color='brown'>(**#**)</font> You're encouraged to think on more features!
* <font color='brown'>(**#**)</font> Pay attention to dimensionality fo the data. For instance, how do you define the _median color_?
* <font color='brown'>(**#**)</font> For simplicity we use the RGB Color Space. Yet color distance might be better calculated in other color spaces (See LAB for instance).

In [ ]:
# Mean Color Per Class
def CalcMeanColorPerClass( mX, vY, imgSize = IMG_SIZE ):
    # Assuming input data is UINT8
    
    vClass = np.unique(vY)
    mColor = np.zeros(shape = (vClass.shape[0], 3)) #<! Each row is a class

    for ii, classIdx in enumerate(vClass):
        numImg = np.sum(vY == classIdx)
        mD = np.reshape(mX[vY == classIdx], (numImg, imgSize[0] * imgSize[1], 3), order = 'F') #<! Data is column stacked
        mColor[ii, :] = np.mean(mD, axis = (0, 1))
    
    return mColor / 255.0

# Mean Color per Image
def CalcMeanColor( vX, imgSize = IMG_SIZE ):

    mI = np.reshape(vX, (imgSize[0] * imgSize[1], 3), order = 'F') #<! Data is column stacked

    return np.mean(mI, axis = 0) / 255.0


In [ ]:
# Mean Histogram per Channel per Class
def CalcRgbHistPerClass( mX, vY, imgSize = IMG_SIZE, lHist = [0, 64, 128, 192, 255]):

    vClass = np.unique(vY)
    tH = np.zeros(shape = (3, len(lHist) - 1, 3)) #<! Color x #Bins x #Classes

    for ii, classIdx in enumerate(vClass):
        numImg = np.sum(vY == classIdx)
        mD = np.reshape(mX[vY == classIdx], (numImg * imgSize[0] * imgSize[1], 3), order = 'F') #<! Data is column stacked
        for jj in range(3):
            # Color Channel
            # mD is ((numImg * imgSize[0] * imgSize[1]) x 3)
            vH, _ = np.histogram(mD[:, jj], bins = lHist)
            tH[jj, :, ii] = vH / np.sum(vH)
    
    return tH

# Histogram per Channel (Single Image)
def CalcHistogram( vX, imgSize = IMG_SIZE, lHist = [0, 64, 128, 192, 255] ):

    mH = np.zeros(shape = (3, len(lHist) - 1))

    mI = np.reshape(vX, (imgSize[0] * imgSize[1], 3), order = 'F') #<! Data is column stacked

    for ii in range(3):
        vH, _ = np.histogram(mI[:, ii], bins = lHist)
        mH[ii] = vH / np.sum(vH)
    
    return mH



In [ ]:
# Channel Value to Mean Value Ratio
# The ratio between the mean value of the channel to the mean value of all pixels
def MeanChannelValueMeanValueRatio(vX, imgSize = IMG_SIZE):
    
    vMeanColor = CalcMeanColor(vX, imgSize = imgSize)
    
    return vMeanColor / np.mean(vMeanColor)

In [ ]:
lFeatureName = ['Red vs. Mean', 'Green vs. Mean', 'Blue vs. Mean', 'Red Channel Hist Distance Class 0', 'Green Channel Hist Distance Class 0', 'Blue Channel Hist Distance Class 0', 'Red Channel Hist Distance Class 1', 'Green Channel Hist Distance Class 1', 'Blue Channel Hist Distance Class 1', 'Red Channel Hist Distance Class 2', 'Green Channel Hist Distance Class 2', 'Blue Channel Hist Distance Class 2', 'Mean Pixel Distance Class 0', 'Mean Pixel Distance Class 1', 'Mean Pixel Distance Class 2']

# Creating the Features Matrix
# The matrix is numSamples x numFeatures
# The features are (15): ratioR, ratioG, ratioB, redHistDisCls0, greenHistDisCls0, blueHistDisCls0, redHistDisCls1, greenHistDisCls1, blueHistDisCls1, redHistDisCls2, greenHistDisCls2, blueHistDisCls2, meanPxDisCls0, meanPxDisCls1, meanPxDisCls2
# Hence the matrix is 2700x15

def CalcFeaturesMatrix( mD, tH, mC ):

    numSamples = mD.shape[0]

    mX = np.zeros(shape = (numSamples, 15))
    
    for ii in range(numSamples):
        vR = MeanChannelValueMeanValueRatio(mD[ii])
        mH = CalcHistogram(mD[ii])
        vC = CalcMeanColor(mD[ii])
        for jj in range(15):
            if jj < 3:
                mX[ii, jj] = vR[jj]
            # The next section could be written in a vectorized manner yet written for clarity
            elif jj == 3:
                mX[ii, jj] = np.linalg.norm(mH[0] - tH[0, :, 0]) #!< Red Channel, Class 0
            elif jj == 4:
                mX[ii, jj] = np.linalg.norm(mH[1] - tH[1, :, 0]) #!< Green Channel, Class 0
            elif jj == 5:
                mX[ii, jj] = np.linalg.norm(mH[2] - tH[2, :, 0]) #!< Blue Channel, Class 0
            elif jj == 6:
                mX[ii, jj] = np.linalg.norm(mH[0] - tH[0, :, 1]) #!< Red Channel, Class 0
            elif jj == 7:
                mX[ii, jj] = np.linalg.norm(mH[1] - tH[1, :, 1]) #!< Green Channel, Class 0
            elif jj == 8:
                mX[ii, jj] = np.linalg.norm(mH[2] - tH[2, :, 1]) #!< Blue Channel, Class 0
            elif jj == 9:
                mX[ii, jj] = np.linalg.norm(mH[0] - tH[0, :, 2]) #!< Red Channel, Class 0
            elif jj == 10:
                mX[ii, jj] = np.linalg.norm(mH[1] - tH[1, :, 2]) #!< Green Channel, Class 0
            elif jj == 11:
                mX[ii, jj] = np.linalg.norm(mH[2] - tH[2, :, 2]) #!< Blue Channel, Class 0
            elif jj == 12:
                mX[ii, jj] = np.linalg.norm(vC - mC[0]) #!<Class 0
            elif jj == 13:
                mX[ii, jj] = np.linalg.norm(vC - mC[1]) #!<Class 1
            elif jj == 14:
                mX[ii, jj] = np.linalg.norm(vC - mC[2]) #!<Class 2
    
    return mX

In [ ]:
# Calculate the Features Matrix for the Training Data Set
mC = CalcMeanColorPerClass(mXTrain, vYTrain) #<! Mean pixel per Class
tH = CalcRgbHistPerClass(mXTrain, vYTrain) #<! Mean histogram per channel per class
mF = CalcFeaturesMatrix(mXTrain, tH, mC) #<! The features matrix

* <font color='brown'>(**#**)</font> One could optimize the histogram by creating a 3D histogram.

### Features Analysis

In this section the relation between the features and the labels is analyzed.  
You should visualize / calculate measures which imply the features makes the classes identifiable.

#### Ideas for Analysis

1. Display the histogram / density of each feature by the label of sample.
2. Display the correlation between the feature to the class value (Pay attention this is a mix of continuous values and categorical values).

* <font color='brown'>(**#**)</font> You may find SeaBorn's `kdeplot()` useful.

In [ ]:
hF, hA = plt.subplots(nrows = 5, ncols = 3, figsize = (12, 8))

for ii, featName in enumerate(lFeatureName):
    sns.kdeplot(x = mF[:, ii], hue = vYTrain, ax = hA.flat[ii])
    hA.flat[ii].set_title(f'Distribution of {featName}')

## Optimize Classifiers

In this section we'll train a Kernel SVM model with optimized hyper parameters: `C` and `gamma`.  
The score should be the regular accuracy.

1. Build the dictionary of parameters for the grid search.
2. Construct the grid search object (`GridSearchCV`).
3. Optimize the hyper parameters by the `fit()` method of the grid search object.

* <font color='red'>(**?**)</font> Why is the accuracy a reasonable score in this case?

In [ ]:
# SciKit Learn requires vY to be matrix form for `_ThresholdScorer()` (See https://github.com/scikit-learn/scikit-learn/blob/7b13a8f120a6d67112b0f50a8834d65e2258f045/sklearn/metrics/_scorer.py#L366)
# Hence we'll use accuracy measure.

# def RocAucSvm( vY, mDecFun ):
    
#     mP      = sp.special.softmax(mDecFun, axis = 1)
#     aucVal  = roc_auc_score(vY, mP, multi_class = 'ovr')
    
#     return aucVal

# RocAucSvmScore = make_scorer(RocAucSvm, needs_threshold = True)

# oGsSvc = GridSearchCV(estimator = SVC(kernel = 'rbf', decision_function_shape = 'ovr'), param_grid = dParams, scoring = RocAucSvmScore, cv = numFold, verbose = 4)

In [ ]:
# Construct the Grid Search object 

#===========================Fill This===========================#
# Set the parameters to iterate over and their values
dParams = {'C': lC, 'gamma': lγ}
#===============================================================#

oGsSvc = GridSearchCV(estimator = SVC(kernel = 'rbf'), param_grid = dParams, scoring = None, cv = numFold, verbose = 4)

In [ ]:
# Optimize

#===========================Fill This===========================#
# Apply the grid search phase
oGsSvc = oGsSvc.fit(mF, vYTrain)
#===============================================================#

## Confusion Matrix on Test Data 

In this section we'll test the model on the test data.

1. Extract the best estimator from the grid search.
2. If needed, fit it to the train data.
3. Calculate the test set features. Make sure to avoid data leakage.
4. Display the _confusion matrix_.

The objective is to get at least `85%` accuracy per class.

In [ ]:
# Extract the Best Model

#===========================Fill This===========================#
# Get the best model with the optimized hyper parameters
bestModel = oGsSvc.best_estimator_
#===============================================================#

* <font color='red'>(**?**)</font> Does the best model need a refit on data?

In [ ]:
# Calculate the test data set features
# Pay attention to not use of leak of data from the test set to the model / features.
# One way to obey this is assume you got the test data one by one.

#===========================Fill This===========================#
# Features of the Test Data
mFTest = CalcFeaturesMatrix(mXTest, tH, mC)
#===============================================================#

In [ ]:
# Plot the Confusion Matrix
hF, hA = plt.subplots(figsize = (10, 10))

#===========================Fill This===========================#
hA, mConfMat = PlotConfusionMatrix(vYTest, bestModel.predict(mFTest), lLabels = L_CLASSES, hA = hA)
#===============================================================#

plt.show()

* <font color='red'>(**?**)</font> If results are good, can you spot the dominant feature for them if there is?
* <font color='green'>(**@**)</font> Check results with a single feature: The channel with the highest mean value.